# 1. Read JSON file

In [6]:
import json

#Read JSON data into the datastore variable
filename = 'online course data.json'
with open(filename, 'r', encoding='UTF-8') as f:
    onlineCourses_JSONarr = json.load(f)

#Use the new datastore datastructure
print(len(onlineCourses_JSONarr))
print(onlineCourses_JSONarr[0].keys())

826
dict_keys(['cTitle', 'cProvider', 'cPhotoLink', 'cDirectLink', 'cDescription', 'cWeekSection', 'cAuthor', 'cLanguage', 'tmpCateg', 'tmpId'])


In [154]:
for i in range(0,10):
    print(onlineCourses_JSONarr[i]['cTitle'])

Machine Learning
用Python玩转数据 Data Processing Using Python
Building a Data Science Team
Command Line Tools for Genomic Data Science
A Crash Course in Data Science
Data Science in Real Life
Materials Data Sciences and Informatics
Computer Science:  Algorithms, Theory, and Machines
Data Science Methodology
Data Science Ethics


In [504]:
str1=str(onlineCourses_JSONarr[114]['cWeekSection'])[2850:2880]
print(str1)
str1=re.sub('\\\\x0b',' ',str1)
str1

Concepts, Processes,\x0band Te


'Concepts, Processes, and Te'

# 2. Helper functions

In [8]:
def addProfessor(prof,courseId):
    '''Check if professor exists in db, creates new professor if not and the TEACHES relationship with course'''
    query = 'MERGE (professor:Professor {name: \"' + prof + '\"})' +\
            'MERGE (course:Course {tmpId: \"' + courseId + '\"})' +\
            'CREATE UNIQUE (professor)-[:TEACHES]->(course)'
    graph.run(query)


In [9]:
def addLanguage(lang ,courseId):
    '''Check if professor exists in db, creates new language if not and the LANGUAGE relationship with course'''
    query = 'MERGE (language:Language {name: \"' + lang + '\"})' +\
            'MERGE (course:Course {tmpId: \"' + courseId + '\"})' +\
            'CREATE UNIQUE (course)-[:LANGUAGE]->(language)'
    graph.run(query)
                

In [10]:
def addCategory(cat ,courseId):
    '''Check if professor exists in db, creates new category if not and the category relationship with course'''
    query = 'MERGE (category:Category {number: \"' + cat + '\"})' +\
            'MERGE (course:Course {tmpId: \"' + courseId + '\"})' +\
            'CREATE UNIQUE (course)-[:IS_CATEGORY]->(category)'
    graph.run(query)
                

In [11]:
def addProvider(prov ,courseId):
    '''Check if professor exists in db, creates new provider if not and the provider relationship with course'''
    query = 'MERGE (provider:Provider {number: \"' + prov + '\"})' +\
            'MERGE (course:Course {tmpId: \"' + courseId + '\"})' +\
            'CREATE UNIQUE (provider)-[:PROVIDES]->(course)'
    graph.run(query)
                

# 3. Connect to Neo4j DB

In [116]:
#connect 
from py2neo import Graph

graph = Graph("http://localhost:7474", user='EvaArevalo', password='yhwh123')

In [117]:
#clean DB
graph.delete_all()

In [118]:
#Create uniqueness constraints
graph.run('CREATE CONSTRAINT ON (professor:Professor) ASSERT professor.name IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (course:Course) ASSERT course.tmpId IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (language:Language) ASSERT language.language IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (category:Category) ASSERT category.number IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (provider:Provider) ASSERT provider.name IS UNIQUE')

In [119]:
#drop constraints if needed
#graph.run('DROP CONSTRAINT ON (course:Course) ASSERT course.title IS UNIQUE')

In [120]:
#Create Nodes and relationships
import re

for course in onlineCourses_JSONarr:
    #Create Course node
    description = ''
    weekSection = ''
    photoLink = ''
    directLink = ''
    if 'cPhotoLink' in course and course['cPhotoLink']!=None:
        photoLink = course['cPhotoLink']
    if 'cDirectLink' in course and course['cDirectLink']!=None:
        directLink = course['cDirectLink']
    if 'cDescription' in course and course['cDescription']!=None:
        description = re.sub('\"', '\'', course['cDescription'])
        description = re.sub('\n\n', ' ', description)
        description = re.sub('\n', ' ', description)
    if 'cWeekSection' in course and course['cWeekSection']!=None:
        weekSection = re.sub('\"', '\'', str(course['cWeekSection']))
        weekSection = re.sub('\n\n', ' ', weekSection)
        weekSection = re.sub('\n', ' ', weekSection)
        weekSection = re.sub('\\\\x0b', ' ', weekSection)
    title = re.sub('\'', '’', course['cTitle'])
    query = 'MERGE (course:Course {tmpId:\"' + str(course['tmpId']) + '\", ' +\
                                    'title:\'' + title + '\', ' +\
                                    'photoLink:\"' + photoLink + '\", ' +\
                                    'directLink:\"' + directLink + '\", ' +\
                                    'description:\"' + description +  '\", ' +\
                                    'weekSection:\"' + weekSection + '\"})' 

    graph.run(query)
    #  'weekSection:\"' + weekSection + '\", ' +\
    
    #Add relationships and other nodes
    addLanguage(course['cLanguage'], str(course['tmpId']))
    addProvider(course['cProvider'], str(course['tmpId']))
    for cat in course['tmpCateg']:
        addCategory(str(cat),str(course['tmpId']))
    for prof in course['cAuthor']:
        profname = re.sub('\"',',',prof)
        addProfessor(profname, str(course['tmpId']))


# 4. Draw graph

In [121]:
import scripts.vis
from scripts.vis import draw

options = {"Professor": "name", "Course": "title", "Language":"language", "Category":"number", "Provider":"name"}
draw(graph, options)

# 5. Dump into GraphJSON format

#json specification
https://github.com/jsongraph/json-graph-specification

In [108]:
#get nodes
edges = []

result = graph.run("MATCH (a)-[r]->(b) WITH collect({source: id(a),target: id(b),caption: type(r)}) AS edges RETURN edges")

for record in result:
    edges+=record['edges']
    
print(edges)

[{'caption': 'IS_CATEGORY', 'source': 4, 'target': 1892}, {'caption': 'LANGUAGE', 'source': 4, 'target': 1743}, {'caption': 'IS_CATEGORY', 'source': 11, 'target': 41}, {'caption': 'LANGUAGE', 'source': 11, 'target': 1743}, {'caption': 'TEACHES', 'source': 12, 'target': 11}, {'caption': 'TEACHES', 'source': 40, 'target': 1894}, {'caption': 'IS_CATEGORY', 'source': 43, 'target': 1892}, {'caption': 'LANGUAGE', 'source': 43, 'target': 1743}, {'caption': 'TEACHES', 'source': 44, 'target': 43}, {'caption': 'IS_CATEGORY', 'source': 97, 'target': 1892}, {'caption': 'LANGUAGE', 'source': 97, 'target': 1743}, {'caption': 'TEACHES', 'source': 116, 'target': 97}, {'caption': 'IS_CATEGORY', 'source': 126, 'target': 1892}, {'caption': 'LANGUAGE', 'source': 126, 'target': 216}, {'caption': 'TEACHES', 'source': 172, 'target': 126}, {'caption': 'IS_CATEGORY', 'source': 231, 'target': 1892}, {'caption': 'LANGUAGE', 'source': 231, 'target': 1743}, {'caption': 'TEACHES', 'source': 271, 'target': 4}, {'cap

In [113]:
#get edges
nodes = []

node_cursors = {}
node_cursors['course'] = graph.run("MATCH (n:Course) WITH collect({id: id(n),type: labels(n)[0],caption:n.title}) AS nodes RETURN nodes")
node_cursors['language'] = graph.run("MATCH (n:Language) WITH collect({id: id(n),type: labels(n)[0],caption:n.language}) AS nodes RETURN nodes")
node_cursors['professor'] = graph.run("MATCH (n:Professor) WITH collect({id: id(n),type: labels(n)[0],caption:n.name}) AS nodes RETURN nodes")
node_cursors['provider'] = graph.run("MATCH (n:Provider) WITH collect({id: id(n),type: labels(n)[0],caption:n.name}) AS nodes RETURN nodes")
node_cursors['category'] =  graph.run("MATCH (n:Category) WITH collect({id: id(n),type: labels(n)[0],caption:''}) AS nodes RETURN nodes")

for key,cursor in node_cursors.items():
    for record in cursor:
        nodes+=record['nodes']

print(nodes)

[{'caption': 'Data Science in Real Life', 'id': 4, 'type': 'Course'}, {'caption': 'Computer Science:  Algorithms, Theory, and Machines', 'id': 11, 'type': 'Course'}, {'caption': 'Data Science Ethics', 'id': 43, 'type': 'Course'}, {'caption': 'Command Line Tools for Genomic Data Science', 'id': 97, 'type': 'Course'}, {'caption': '用Python玩转数据 Data Processing Using Python', 'id': 126, 'type': 'Course'}, {'caption': 'Building a Data Science Team', 'id': 231, 'type': 'Course'}, {'caption': 'A Crash Course in Data Science', 'id': 272, 'type': 'Course'}, {'caption': 'Machine Learning', 'id': 1670, 'type': 'Course'}, {'caption': 'Data Science Methodology', 'id': 1810, 'type': 'Course'}, {'caption': 'Materials Data Sciences and Informatics', 'id': 1894, 'type': 'Course'}, {'caption': None, 'id': 216, 'type': 'Language'}, {'caption': None, 'id': 1743, 'type': 'Language'}, {'caption': 'Robert Sedgewick', 'id': 12, 'type': 'Professor'}, {'caption': 'Dr. Surya Kalidindi', 'id': 40, 'type': 'Profess

In [114]:
#build graph and dump to GraphJSON
import json

node_graph = {}
node_graph['nodes'] = nodes
node_graph['edges'] = edges

print(json.dumps(node_graph))

with open("data/courses.json", "w") as f:
    json.dump(node_graph, f)

{"nodes": [{"caption": "Data Science in Real Life", "id": 4, "type": "Course"}, {"caption": "Computer Science:  Algorithms, Theory, and Machines", "id": 11, "type": "Course"}, {"caption": "Data Science Ethics", "id": 43, "type": "Course"}, {"caption": "Command Line Tools for Genomic Data Science", "id": 97, "type": "Course"}, {"caption": "\u7528Python\u73a9\u8f6c\u6570\u636e Data Processing Using Python", "id": 126, "type": "Course"}, {"caption": "Building a Data Science Team", "id": 231, "type": "Course"}, {"caption": "A Crash Course in Data Science", "id": 272, "type": "Course"}, {"caption": "Machine Learning", "id": 1670, "type": "Course"}, {"caption": "Data Science Methodology", "id": 1810, "type": "Course"}, {"caption": "Materials Data Sciences and Informatics", "id": 1894, "type": "Course"}, {"caption": null, "id": 216, "type": "Language"}, {"caption": null, "id": 1743, "type": "Language"}, {"caption": "Robert Sedgewick", "id": 12, "type": "Professor"}, {"caption": "Dr. Surya Kal